# How to read data

Use JumpPoseDataset in dataset.py to create a dataset that iterates over jump (xyzt) data and its label data as a tuple.

In [1]:
import sys

sys.path.append('..')
from dataset.dataset import JumpPoseDataset


In [2]:
pose_paths = '../dataset/data'
info_path = '../dataset/info.xlsx'
dataset = JumpPoseDataset(pose_paths, info_path)
pose, label = dataset.get_example(0)

In [3]:
labe

(1137, 88)